In [1]:
import numpy as np
import pandas as pd

import bentoml

import json

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier

C:\Users\daver\miniconda3\envs\tf_gpu\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
url = "https://raw.githubusercontent.com/DonAvery/health-insurance-cross-sell-prediction/main/data/cleaned-insurance-cross-sell.csv"
df = pd.read_csv(url)

In [3]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.response == 'default').astype('int').values
y_test = (df_test.response == 'default').astype('int').values

del df_train['response']
del df_test['response']

In [4]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [5]:
rf = RandomForestClassifier(n_estimators=400, max_depth=10, min_samples_leaf=3, random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=400,
                       random_state=1)

In [6]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [7]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1
}

model = xgb.train(xgb_params, dtrain,
                  num_boost_round=470)

Code to use for our test of the model.

In [8]:
request = df_test.iloc[0].to_dict()
print(json.dumps(request, indent=2))

{
  "id": 61311,
  "gender": "male",
  "age": 68,
  "driving_license": 1,
  "region_code": 28.0,
  "previously_insured": 1,
  "vehicle_age": "1-2 year",
  "vehicle_damage": "no",
  "annual_premium": 32788.0,
  "policy_sales_channel": 26.0,
  "vintage": 31
}
